# Visualización mejorada de la segmentación urbana-rural-vegetación

Este notebook es una continuación del análisis de segmentación urbana realizado en el notebook `01_exploration.ipynb`. Aquí nos centraremos específicamente en mejorar las visualizaciones de la segmentación de áreas urbanas, rurales y vegetación, asegurando que los elementos visuales estén bien distribuidos y no se superpongan.

In [ ]:
# 1. Importar librerías necesarias
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
from skimage.feature import local_binary_pattern
from skimage import color
import cv2
import warnings
warnings.filterwarnings('ignore')

# Asegurarse que las figuras se muestran con alta resolución
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

print("✅ Librerías importadas correctamente (incluyendo scikit-image y cv2)")

## Configuración inicial y carga de datos

En esta sección, vamos a configurar las rutas necesarias y cargar los datos generados en el notebook anterior. Primero necesitamos cargar los resultados de segmentación desde el notebook original para poder trabajar con ellos.

In [ ]:
# 2. Configuración y Pipeline de Segmentación Avanzada con Post-procesamiento

# --- Parte 1: Configuración de Rutas y Carga de Datos ---
from pathlib import Path
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy import ndimage
from scipy.ndimage import median_filter
from skimage import morphology

# Configurar rutas
project_root = Path(os.getcwd()).parent
data_path = project_root / "data"
raw_data_path = data_path / "raw"
results_path = project_root / "results"
figures_path = project_root / "figures"

# Crear directorios si no existen
results_path.mkdir(exist_ok=True)
figures_path.mkdir(exist_ok=True)

print(f"✅ Rutas configuradas:")
print(f"  • Datos sin procesar: {raw_data_path}")

# Listar imágenes disponibles
image_files = sorted([f for f in raw_data_path.glob("*.jpg")])
print(f"\n📁 Imágenes disponibles: {len(image_files)} archivos")
for img_file in image_files[:5]:
    print(f"  • {img_file.name}")
if len(image_files) > 5:
    print(f"  • ... y {len(image_files) - 5} más")

# Función para cargar imagen (no cambia)
def cargar_imagen(imagen_path, target_size=(400, 400)):
    """Carga una imagen y la redimensiona al tamaño objetivo"""
    try:
        img = Image.open(imagen_path)
        img = img.convert('RGB')
        img = img.resize(target_size)
        return np.array(img) / 255.0
    except Exception as e:
        print(f"Error cargando {imagen_path}: {e}")
        return None

# --- Parte 2: Pipeline de Segmentación Avanzada (Inspirado en SLEUTH) ---

def extraer_features_hibridas(img_rgb):
    """
    Extrae un conjunto SELECTIVO de características que realmente aportan valor.
    Features: [R, G, B, NDVI] - Solo 4 características bien elegidas
    """
    r = img_rgb[:, :, 0].astype(float)
    g = img_rgb[:, :, 1].astype(float)
    b = img_rgb[:, :, 2].astype(float)
    
    # NDVI aproximado - LA ÚNICA característica adicional que realmente aporta
    epsilon = 1e-6
    ndvi = (g - r) / (g + r + epsilon)
    
    # Apilar SOLO las características que funcionan
    features = np.stack([r, g, b, ndvi], axis=-1)
    
    return features

def segmentar_con_metodo_hibrido(img_rgb, usar_solo_rgb=False):
    """
    Método híbrido MEJORADO: RGB como base + NDVI selectivo + parámetros optimizados
    """
    h, w, _ = img_rgb.shape
    
    if usar_solo_rgb:
        # Método RGB puro con escalado cuidadoso
        pixels = img_rgb.reshape(-1, 3)
        scaler = StandardScaler()
        pixels_scaled = scaler.fit_transform(pixels)
        n_features = 3
    else:
        # Método híbrido con NDVI pero escalado balanceado
        features = extraer_features_hibridas(img_rgb)
        pixels = features.reshape(-1, features.shape[-1])
        
        # Escalado más cuidadoso - darle más peso a RGB
        scaler = StandardScaler()
        pixels_scaled = scaler.fit_transform(pixels)
        
        # Rebalancear: RGB tiene peso normal, NDVI peso reducido
        pixels_scaled[:, 3] = pixels_scaled[:, 3] * 0.7  # Reducir influencia del NDVI
        n_features = 4
    
    # K-means con parámetros optimizados para mejor convergencia
    kmeans = KMeans(
        n_clusters=3, 
        random_state=42, 
        n_init=25,  # Más intentos
        max_iter=400,  # Más iteraciones
        tol=1e-6,  # Mayor precisión
        algorithm='elkan'  # Algoritmo más eficiente
    )
    
    labels = kmeans.fit_predict(pixels_scaled)
    labels = labels.reshape(h, w)
    
    # Centroides en espacio original
    centroides = scaler.inverse_transform(kmeans.cluster_centers_)
    
    # Confianza mejorada
    distances = kmeans.transform(pixels_scaled)
    min_distances = np.min(distances, axis=1)
    max_distance = np.max(min_distances)
    
    # Confianza normalizada más suave
    confidence = 1.0 - (min_distances / (max_distance + 1e-6))
    confidence = np.clip(confidence, 0.3, 1.0)  # Limitar rango para mejor visualización
    confidence = confidence.reshape(h, w)
    
    return labels, centroides, confidence, n_features

def post_procesar_segmentacion(labels, confidence, ventana=5):
    """
    Post-procesamiento MEJORADO que preserva estructura urbana y mejora conectividad
    """
    h, w = labels.shape
    labels_mejorados = labels.copy()
    
    # Máscara de alta confianza - estas áreas no se modifican
    alta_confianza = confidence > 0.7
    
    # 1. Filtro de mediana SELECTIVO - solo en áreas de baja confianza
    mascara_filtrar = confidence < 0.5
    if np.any(mascara_filtrar):
        labels_filtrados = median_filter(labels_mejorados, size=3)
        labels_mejorados = np.where(mascara_filtrar, labels_filtrados, labels_mejorados)
    
    # 2. Tratamiento específico por clase
    for clase in [1, 2, 3]:
        mask_clase = (labels_mejorados == clase)
        
        if np.any(mask_clase):
            # Parámetros adaptativos por clase
            if clase == 1:  # Urbano - preservar detalles, mejorar conectividad
                min_size = 30  # Menor tamaño mínimo para capturar detalles urbanos
                hole_size = 40
                kernel_size = 7  # Kernel más grande para conectar áreas urbanas
                umbral_mayoria = 0.4  # Menos estricto para áreas urbanas
                
                # Operación especial: conectar áreas urbanas cercanas
                kernel_conectar = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
                mask_expandida = cv2.morphologyEx(mask_clase.astype(np.uint8), 
                                                cv2.MORPH_CLOSE, kernel_conectar)
                
                # Solo aplicar conexión donde hay evidencia (confianza media)
                mask_aplicar = confidence > 0.4
                mask_clase = np.where(mask_aplicar & (mask_expandida == 1), True, mask_clase)
                
            elif clase == 2:  # Vegetación - preservar formas naturales
                min_size = 40
                hole_size = 25
                kernel_size = 5
                umbral_mayoria = 0.5
                
            else:  # Rural - operaciones más amplias
                min_size = 60
                hole_size = 50
                kernel_size = 7
                umbral_mayoria = 0.6
            
            # Aplicar limpieza morfológica
            mask_limpia = morphology.remove_small_objects(mask_clase, min_size=min_size)
            mask_limpia = morphology.remove_small_holes(mask_limpia, area_threshold=hole_size)
            
            # Solo aplicar cambios en áreas de baja/media confianza
            mask_cambios = confidence < 0.8
            labels_mejorados = np.where(mask_cambios & mask_limpia, clase, 
                                      np.where(mask_cambios & mask_clase & ~mask_limpia, 0, labels_mejorados))
    
    # 3. Filtro de mayoría local ADAPTATIVO
    for clase in [1, 2, 3]:
        mask = (labels_mejorados == clase).astype(float)
        kernel = np.ones((ventana, ventana))
        vecinos = ndimage.convolve(mask, kernel, mode='constant')
        
        # Umbral adaptativo por clase
        if clase == 1:  # Urbano - menos restrictivo
            threshold = ventana * ventana * 0.35
        elif clase == 2:  # Vegetación
            threshold = ventana * ventana * 0.45
        else:  # Rural
            threshold = ventana * ventana * 0.55
        
        strong_regions = vecinos > threshold
        
        # Solo aplicar en áreas de baja confianza
        mask_aplicar = confidence < 0.6
        strong_regions = strong_regions & mask_aplicar
        
        labels_mejorados[strong_regions] = clase
    
    # 4. Reasignación inteligente de píxeles sin asignar
    pixels_sin_asignar = (labels_mejorados == 0)
    if np.any(pixels_sin_asignar):
        for i in range(h):
            for j in range(w):
                if labels_mejorados[i, j] == 0:
                    # Ventana adaptativa según la confianza local
                    radio = 3 if confidence[i, j] < 0.3 else 2
                    i_min, i_max = max(0, i-radio), min(h, i+radio+1)
                    j_min, j_max = max(0, j-radio), min(w, j+radio+1)
                    
                    vecindario = labels_mejorados[i_min:i_max, j_min:j_max]
                    vecindario_valido = vecindario[vecindario > 0]
                    
                    if len(vecindario_valido) > 0:
                        # Considerar confianza del vecindario
                        confianza_vecindario = confidence[i_min:i_max, j_min:j_max]
                        peso_confianza = confianza_vecindario[vecindario > 0]
                        
                        # Voto ponderado por confianza
                        clases_unicas = np.unique(vecindario_valido)
                        mejor_clase = clases_unicas[0]
                        mejor_peso = 0
                        
                        for clase_cand in clases_unicas:
                            peso_clase = np.sum(peso_confianza[vecindario_valido == clase_cand])
                            if peso_clase > mejor_peso:
                                mejor_peso = peso_clase
                                mejor_clase = clase_cand
                        
                        labels_mejorados[i, j] = mejor_clase
                    else:
                        labels_mejorados[i, j] = 3  # Por defecto, rural
    
    return labels_mejorados

def clasificar_hibrido_inteligente(centroides, img_rgb, n_features):
    """
    Clasificación híbrida CORREGIDA - Volviendo a criterios más precisos
    """
    clasificacion = {}
    print("🧠 Análisis híbrido RGB + NDVI (versión corregida):")
    
    # Calcular estadísticas globales más conservadoras
    r_global = img_rgb[:, :, 0].flatten()
    g_global = img_rgb[:, :, 1].flatten()
    b_global = img_rgb[:, :, 2].flatten()
    
    # Umbrales más restrictivos
    brillo_percentil_alto = np.percentile((r_global + g_global + b_global) / 3, 85)  # Más alto
    verdor_percentil_alto = np.percentile(g_global - (r_global + b_global) / 2, 80)  # Más alto
    
    print(f"  📊 Umbrales conservadores: Brillo>{brillo_percentil_alto:.3f}, Verdor>{verdor_percentil_alto:.3f}")
    
    for i, centroide in enumerate(centroides):
        r, g, b = centroide[0], centroide[1], centroide[2]
        
        # Calcular características
        brillo = (r + g + b) / 3
        verdor_rgb = g - (r + b) / 2
        saturacion_rgb = np.std([r, g, b])
        
        if n_features > 3:
            ndvi = centroide[3]
            print(f"  Cluster {i}: R={r:.3f}, G={g:.3f}, B={b:.3f}, NDVI={ndvi:.3f}")
            print(f"             Brillo={brillo:.3f}, Verdor={verdor_rgb:.3f}, Saturación={saturacion_rgb:.3f}")
        else:
            ndvi = (g - r) / (g + r + 0.001)
            print(f"  Cluster {i}: R={r:.3f}, G={g:.3f}, B={b:.3f}, NDVI_calc={ndvi:.3f}")
            print(f"             Brillo={brillo:.3f}, Verdor={verdor_rgb:.3f}, Saturación={saturacion_rgb:.3f}")
        
        # LÓGICA DE CLASIFICACIÓN BALANCEADA
        
        # 1. Vegetación: Criterios más flexibles pero aún precisos
        if (verdor_rgb > 0.035 and ndvi > 0.08 and g > r) or (ndvi > 0.15):
            clase = 2
            tipo = "Vegetación (Verde + NDVI)"
            
        # 2. Urbano: Criterios específicos pero no demasiado restrictivos
        elif (
            # Criterio 1: Colores claros (edificios brillantes)
            brillo > brillo_percentil_alto or
            # Criterio 2: Grises neutros específicos (infraestructura)
            (abs(r - g) < 0.06 and abs(g - b) < 0.06 and brillo > 0.42) or
            # Criterio 3: Baja saturación con brillo alto (concreto/asfalto)
            (saturacion_rgb < 0.1 and brillo > 0.47)
        ):
            clase = 1
            tipo = "Urbano (Infraestructura)"
            
        # 3. Rural: Todo lo demás
        else:
            clase = 3
            tipo = "Rural (Tierra, mixto)"
        
        clasificacion[i] = clase
        print(f"    → {tipo}")
    
    # REBALANCEO INTELIGENTE - Asegurar mínima representación
    clases_presentes = set(clasificacion.values())
    print(f"  📊 Clases detectadas inicialmente: {sorted(clases_presentes)}")
    
    # Si falta vegetación, buscar el cluster más verde
    if 2 not in clases_presentes:
        print("  🌱 Buscando vegetación...")
        clusters_info = []
        for i, centroide in enumerate(centroides):
            r, g, b = centroide[0], centroide[1], centroide[2]
            verdor = g - (r + b) / 2
            ndvi_val = centroide[3] if n_features > 3 else (g - r) / (g + r + 0.001)
            
            clusters_info.append({
                'id': i,
                'verdor': verdor,
                'ndvi': ndvi_val,
                'g_value': g
            })
        
        # Buscar el cluster más verde que no sea urbano
        candidatos_vegetacion = [c for c in clusters_info if clasificacion[c['id']] != 1]
        if candidatos_vegetacion:
            mejor_vegetacion = max(candidatos_vegetacion, key=lambda x: x['ndvi'])
            if mejor_vegetacion['ndvi'] > 0.02:  # Mínimo umbral
                clasificacion[mejor_vegetacion['id']] = 2
                print(f"    → Cluster {mejor_vegetacion['id']} → Vegetación (NDVI: {mejor_vegetacion['ndvi']:.3f})")
    
    # Si solo hay una clase, forzar diversidad mínima
    if len(clases_presentes) == 1:
        print("  🔧 Forzando diversidad mínima...")
        
        clusters_info = []
        for i, centroide in enumerate(centroides):
            r, g, b = centroide[0], centroide[1], centroide[2]
            brillo = (r + g + b) / 3
            verdor = g - (r + b) / 2
            ndvi_val = centroide[3] if n_features > 3 else (g - r) / (g + r + 0.001)
            
            clusters_info.append({
                'id': i,
                'brillo': brillo,
                'verdor': verdor,
                'ndvi': ndvi_val
            })
        
        clusters_ordenados = sorted(clusters_info, key=lambda x: x['brillo'])
        
        # Asignar diversidad básica
        if len(clusters_ordenados) >= 3:
            clasificacion[clusters_ordenados[0]['id']] = 3  # Más oscuro = Rural
            clasificacion[clusters_ordenados[1]['id']] = 2  # Intermedio = Vegetación
            clasificacion[clusters_ordenados[2]['id']] = 1  # Más claro = Urbano
        elif len(clusters_ordenados) == 2:
            clasificacion[clusters_ordenados[0]['id']] = 3  # Rural
            clasificacion[clusters_ordenados[1]['id']] = 1  # Urbano
    
    print(f"  ✅ Clasificación final: {sorted(clasificacion.items())}")
    return clasificacion

def post_procesar_suave(labels, ventana=3):
    """
    Post-procesamiento MÁS SUAVE que preserva más detalles urbanos
    """
    labels_mejorados = labels.copy()
    
    # 1. Solo filtro de mediana MUY suave para eliminar píxeles aislados
    labels_mejorados = median_filter(labels_mejorados, size=2)
    
    # 2. Eliminar SOLO objetos muy pequeños (menos de 20 píxeles)
    for clase in [1, 2, 3]:
        mask = (labels_mejorados == clase)
        mask_limpia = morphology.remove_small_objects(mask, min_size=20)
        # Solo cambiar píxeles de objetos MUY pequeños
        pixels_a_cambiar = mask & ~mask_limpia
        
        # Reasignar a la clase más común en el vecindario inmediato
        coords = np.where(pixels_a_cambiar)
        for i, j in zip(coords[0], coords[1]):
            # Vecindario 3x3
            i_min, i_max = max(0, i-1), min(labels.shape[0], i+2)
            j_min, j_max = max(0, j-1), min(labels.shape[1], j+2)
            vecindario = labels_mejorados[i_min:i_max, j_min:j_max]
            vecindario_valido = vecindario[vecindario != labels_mejorados[i, j]]
            
            if len(vecindario_valido) > 0:
                nueva_clase = np.bincount(vecindario_valido).argmax()
                labels_mejorados[i, j] = nueva_clase
    
    return labels_mejorados

def ejecutar_pipeline_hibrido(imagen_path, usar_solo_rgb=False):
    """
    Pipeline híbrido optimizado: RGB + NDVI selectivo + post-procesamiento suave
    """
    print(f"\n🖼️  Pipeline Híbrido: {imagen_path.name}")
    print(f"     Modo: {'Solo RGB' if usar_solo_rgb else 'RGB + NDVI selectivo'}")
    
    img_rgb = cargar_imagen(imagen_path)
    if img_rgb is None:
        return None
    
    # Segmentación híbrida
    labels, centroides, confidence, n_features = segmentar_con_metodo_hibrido(img_rgb, usar_solo_rgb)
    
    # Clasificación inteligente
    clasificacion = clasificar_hibrido_inteligente(centroides, img_rgb, n_features)
    
    # Mapear etiquetas
    labels_semanticos = np.zeros_like(labels)
    for cluster_id, clase_id in clasificacion.items():
        labels_semanticos[labels == cluster_id] = clase_id
    
    # Post-procesamiento con función mejorada (no el suave que es muy básico)
    print("🔧 Post-procesamiento inteligente...")
    labels_finales = post_procesar_segmentacion(labels_semanticos, confidence)
    
    print("✅ Pipeline híbrido completado.")
    
    return {
        'rgb': img_rgb, 'labels': labels_finales, 'confidence': confidence,
        'year': int(imagen_path.stem.split('_')[-1]), 'method': 'RGB' if usar_solo_rgb else 'Híbrido'
    }

# --- Ejecución de demostración ---

# Definir las clases y colores (esto no cambia)
class_names = ["Urbano", "Vegetación", "Rural"]
class_colors = [
    [0.8, 0.1, 0.1],  # Rojo para áreas urbanas
    [0.1, 0.6, 0.1],  # Verde para vegetación
    [0.8, 0.7, 0.3]   # Beige para rural
]

# --- Comparación de métodos ---

imagen_seleccionada = image_files[-1]  # Año 2004

print("🔬 COMPARACIÓN DE MÉTODOS:")
print("="*50)

# Método 1: Solo RGB (el que funcionaba bien originalmente)
resultado_rgb = ejecutar_pipeline_hibrido(imagen_seleccionada, usar_solo_rgb=True)

# Método 2: Híbrido RGB + NDVI selectivo
resultado_hibrido = ejecutar_pipeline_hibrido(imagen_seleccionada, usar_solo_rgb=False)

# Mostrar comparación
if resultado_rgb and resultado_hibrido:
    print("\n📊 COMPARACIÓN DE RESULTADOS:")
    print("="*60)
    print(f"{'Método':<15} {'Urbano%':<10} {'Vegetación%':<12} {'Rural%':<10}")
    print("="*60)
    
    for resultado, nombre in [(resultado_rgb, 'Solo RGB'), (resultado_hibrido, 'RGB+NDVI')]:
        stats = {}
        for i, name in enumerate(class_names, 1):
            count = np.sum(resultado['labels'] == i)
            percentage = 100.0 * count / resultado['labels'].size
            stats[name.lower()] = percentage
        
        print(f"{nombre:<15} {stats['urbano']:<10.1f} {stats['vegetación']:<12.1f} {stats['rural']:<10.1f}")
    
    print("="*60)
    
    # Usar el mejor resultado para visualización
    print("\n🎯 Seleccionando el mejor resultado...")
    
    # Criterio: preferir el que detecte vegetación Y tenga distribución más equilibrada
    rgb_veg = sum(resultado_rgb['labels'].flatten() == 2) / resultado_rgb['labels'].size
    hyb_veg = sum(resultado_hibrido['labels'].flatten() == 2) / resultado_hibrido['labels'].size
    
    if hyb_veg > rgb_veg and hyb_veg > 0.05:  # Al menos 5% de vegetación detectada
        mejor_resultado = resultado_hibrido
        print("→ Método Híbrido RGB+NDVI seleccionado (mejor detección de vegetación)")
    else:
        mejor_resultado = resultado_rgb
        print("→ Método RGB puro seleccionado (más confiable)")
    
    # Actualizar variables globales
    sample_rgb = mejor_resultado['rgb']
    sample_labels = mejor_resultado['labels']
    sample_confidence = mejor_resultado['confidence']
    
    print(f"\n📊 Distribución final ({mejor_resultado['method']}):")
    for i, name in enumerate(class_names, 1):
        count = np.sum(sample_labels == i)
        percentage = 100.0 * count / sample_labels.size
        print(f"    - {name}: {count:,} píxeles ({percentage:.1f}%)")

## Visualización mejorada de la segmentación

Ahora vamos a crear una visualización mejorada que muestre claramente la segmentación urbano-rural-vegetación. Vamos a asegurarnos de que todos los elementos estén bien organizados y no se superpongan, utilizando un layout más espacioso y claro.

In [ ]:
# 3. Visualización mejorada de la segmentación urbano-rural-vegetación

def crear_mapa_coloreado(labels, colors):
    """Crea un mapa RGB a partir de etiquetas y colores"""
    h, w = labels.shape
    colored_map = np.zeros((h, w, 3))
    for i, color in enumerate(colors, 1):
        mask = (labels == i)
        colored_map[mask] = color
    return colored_map

# Crear el mapa coloreado
urban_rural_map = crear_mapa_coloreado(sample_labels, class_colors)

# Configuración de la figura con mayor espacio y mejor distribución
plt.figure(figsize=(22, 16))

# Crear un grid más espacioso para evitar superposiciones
gs = gridspec.GridSpec(3, 4, height_ratios=[2, 2, 1], width_ratios=[1, 1, 1, 1])

# 1. Mapa segmentado completo (panel grande a la izquierda)
ax1 = plt.subplot(gs[0:2, 0:2])
ax1.imshow(urban_rural_map)
ax1.set_title("Segmentación urbano-rural-vegetación", fontsize=18, fontweight='bold')
ax1.axis('off')

# 2. Imagen original RGB (arriba a la derecha)
ax2 = plt.subplot(gs[0, 2])
ax2.imshow(sample_rgb)
ax2.set_title("Imagen original (RGB)", fontsize=16, fontweight='bold')
ax2.axis('off')

# 3. Mapa de confianza (centro a la derecha)
ax3 = plt.subplot(gs[0, 3])
confidence_vis = ax3.imshow(sample_confidence, cmap='RdYlGn', vmin=0.5, vmax=1.0)
ax3.set_title("Mapa de confianza", fontsize=16, fontweight='bold')
ax3.axis('off')

# Agregar colorbar para el mapa de confianza
divider = make_axes_locatable(ax3)
cax = divider.append_axes("right", size="5%", pad=0.05)
cbar = plt.colorbar(confidence_vis, cax=cax, orientation='vertical')
cbar.set_label('Nivel de confianza', fontsize=12)

# 4. Superposición con transparencia (abajo a la derecha)
ax4 = plt.subplot(gs[1, 2:])
overlay = urban_rural_map.copy()
alpha = 0.6  # Transparencia
for i in range(3):
    overlay[:,:,i] = overlay[:,:,i] * alpha + sample_rgb[:,:,i] * (1-alpha)
ax4.imshow(overlay)
ax4.set_title("Superposición de clases sobre imagen", fontsize=16, fontweight='bold')
ax4.axis('off')

# 5. Panel de estadísticas y gráfico de pastel (abajo)
ax5 = plt.subplot(gs[2, 0:2])
ax5.axis('off')

# Crear DataFrame para estadísticas
stats_data = []
for i, name in enumerate(class_names, 1):
    count = np.sum(sample_labels == i)
    percentage = 100.0 * count / sample_labels.size
    avg_conf = np.mean(sample_confidence[sample_labels == i])
    stats_data.append({
        'Clase': name,
        'Píxeles': count,
        'Porcentaje': percentage,
        'Confianza': avg_conf
    })

df_stats = pd.DataFrame(stats_data)

# Crear tabla de estadísticas
cell_text = []
for i, row in df_stats.iterrows():
    cell_text.append([
        row['Clase'], 
        f"{row['Píxeles']:,}", 
        f"{row['Porcentaje']:.1f}%", 
        f"{row['Confianza']:.3f}"
    ])

ax5.table(
    cellText=cell_text,
    colLabels=['Clase', 'Píxeles', 'Porcentaje', 'Confianza Media'],
    loc='center',
    cellLoc='center',
    bbox=[0.1, 0.3, 0.8, 0.5]  # [left, bottom, width, height]
)
ax5.set_title("Estadísticas de clases", fontsize=16, fontweight='bold')

# 6. Gráfico de pastel (abajo a la derecha)
ax6 = plt.subplot(gs[2, 2:])

# Obtener valores para el gráfico
sizes = df_stats['Porcentaje'].values
explode = (0.1, 0.0, 0.0)  # Explotar la porción urbana

wedges, texts, autotexts = ax6.pie(
    sizes, 
    explode=explode,
    labels=class_names, 
    colors=class_colors,
    autopct='%1.1f%%',
    shadow=True,
    startangle=90,
    textprops={'fontsize': 14},
    wedgeprops={'edgecolor': 'w', 'linewidth': 1}
)

for text in texts + autotexts:
    text.set_fontweight('bold')

ax6.set_title('Distribución de coberturas', fontsize=16, fontweight='bold')

# Crear leyenda separada con muestras de colores más grandes
legend_elements = []
for i, name in enumerate(class_names):
    legend_elements.append(
        Patch(facecolor=class_colors[i], 
              edgecolor='black', 
              label=f"{name} ({df_stats['Porcentaje'].values[i]:.1f}%)")
    )

# Añadir leyenda en un lugar separado para evitar superposiciones
legend = plt.figlegend(
    handles=legend_elements,
    loc='upper center',
    bbox_to_anchor=(0.5, 0.06),
    fontsize=14,
    frameon=True,
    ncol=3
)

# Título global
plt.suptitle("SEGMENTACIÓN URBANO-RURAL-VEGETACIÓN", 
             fontsize=22, fontweight='bold', y=0.98)

plt.tight_layout(rect=[0, 0.08, 1, 0.95])  # Ajuste para evitar superposiciones
plt.subplots_adjust(wspace=0.3, hspace=0.3)  # Añadir más espacio entre subplots

plt.show()

# Guardar la figura en alta resolución
plt.savefig(figures_path / "segmentacion_urbano_rural_vegetacion.png", dpi=300, bbox_inches='tight')
print("\n✅ Visualización mejorada completada y guardada en el directorio 'figures'")
print("La visualización muestra claramente las diferentes categorías sin elementos superpuestos.")

## 🧬 Explicación del Pipeline de Segmentación Híbrida

El método que hemos desarrollado es un pipeline híbrido y adaptativo diseñado para maximizar la precisión en la clasificación de coberturas de suelo. A continuación se detalla cada paso del proceso:

### 1. Carga y Pre-procesamiento de Imagen
- **Entrada**: Imagen satelital en formato JPG.
- **Proceso**: La imagen se carga y se convierte al espacio de color RGB. Para estandarizar el análisis, todas las imágenes se redimensionan a un tamaño uniforme de 400x400 píxeles y sus valores de color se normalizan en un rango de 0 a 1.

### 2. Extracción de Características Híbridas
En lugar de usar solo los canales RGB, creamos un conjunto de "características" más rico para cada píxel, lo que permite al algoritmo tomar decisiones más informadas.
- **Canales RGB**: Los valores normalizados de Rojo, Verde y Azul son las características base.
- **NDVI (Índice de Vegetación de Diferencia Normalizada)**: Calculamos una aproximación del NDVI usando la fórmula `(Verde - Rojo) / (Verde + Rojo)`. Esta es la característica más importante para discriminar vegetación de otras superficies.
- **Resultado**: Cada píxel se convierte en un vector de 4 dimensiones: `[R, G, B, NDVI]`.

### 3. Clustering con K-Means Optimizado
El objetivo es agrupar los 160,000 píxeles de la imagen en 3 grupos (clusters) basándose en sus características.
- **Algoritmo**: Se utiliza K-Means con parámetros optimizados para mayor estabilidad y precisión (`n_init=25`, `max_iter=400`, `algorithm='elkan'`).
- **Balance de Características**: Para evitar que el NDVI domine la clasificación (lo que podría causar que todo se clasifique como vegetación o no), su influencia se reduce ligeramente (`peso = 0.7`). Esto permite que las características de color RGB sigan siendo el factor principal para áreas no vegetales.
- **Resultado**: Se obtienen 3 clusters y sus "centroides", que son los vectores de características promedio para cada grupo.

### 4. Clasificación Semántica Adaptativa
Este es el "cerebro" del sistema. Asigna una etiqueta (Urbano, Vegetación, Rural) a cada uno de los 3 clusters.
- **Umbrales Adaptativos**: En lugar de usar valores fijos, el sistema primero analiza la distribución global de brillo y "verdor" de la imagen para calcular umbrales dinámicos. Esto hace que el método sea robusto ante cambios de iluminación entre diferentes años.
- **Lógica de Decisión Multi-criterio**:
  - **Vegetación**: Se asigna si un cluster tiene un NDVI y un nivel de "verdor" suficientemente altos.
  - **Urbano**: Se utiliza un conjunto de reglas para identificar infraestructura. Un cluster se considera urbano si es muy brillante (techos, concreto), si es un gris neutro (asfalto) o si tiene un brillo alto con baja saturación de color.
  - **Rural**: Es la clase por defecto para todo lo que no es claramente urbano ni vegetación (tierra, roca, vegetación seca).
- **Rebalanceo de Emergencia**: Si la lógica inicial no logra encontrar una de las clases (por ejemplo, en una imagen muy árida donde no se detecta vegetación), un sistema de seguridad busca el cluster "más verde" disponible y lo asigna a Vegetación para asegurar que las 3 clases estén siempre presentes.

### 5. Post-procesamiento Inteligente y Contextual
La clasificación inicial puede tener "ruido" (píxeles aislados mal clasificados). El post-procesamiento lo limpia de forma inteligente.
- **Filtro Basado en Confianza**: Se calcula un "mapa de confianza" para cada píxel. Las operaciones de limpieza más agresivas solo se aplican en áreas donde el algoritmo tiene baja confianza, preservando así los detalles finos en áreas bien definidas.
- **Morfología Adaptativa por Clase**: Se aplican diferentes técnicas de limpieza para cada clase. Por ejemplo, para la clase "Urbano", se utilizan operaciones que tienden a conectar áreas cercanas para formar calles o barrios coherentes, mientras que para "Vegetación" se preservan mejor las formas orgánicas.
- **Reasignación Ponderada**: Los píxeles que se eliminan por ser "ruido" no se descartan, sino que se reasignan a la clase más probable basándose en un voto ponderado por la confianza de sus vecinos.

### 6. Visualización y Selección del Mejor Método
Finalmente, el pipeline se ejecuta dos veces: una en modo "Solo RGB" y otra en modo "Híbrido".
- **Comparación**: Se comparan los resultados de ambos métodos.
- **Selección Automática**: El sistema selecciona automáticamente el resultado del método híbrido si este logra detectar una cantidad razonable de vegetación (>5%). De lo contrario, se queda con el resultado del método "Solo RGB", que es más conservador y confiable.
- **Resultado Final**: Una imagen de segmentación limpia, precisa y contextualizada, junto con estadísticas detalladas de la distribución de clases.

In [ ]:
# 5. Procesamiento y Visualización en Lote de Todas las Imágenes

import time
from datetime import datetime

def procesar_y_guardar_resultados(años_seleccionados=None, mostrar_progreso=True):
    """
    Procesa una selección de imágenes, guarda los resultados y devuelve los datos.
    """
    imagenes_a_procesar = image_files
    if años_seleccionados is not None:
        imagenes_a_procesar = [f for f in image_files if any(str(año) in f.name for año in años_seleccionados)]
    
    print(f"🚀 Iniciando procesamiento de {len(imagenes_a_procesar)} imágenes...")
    
    resultados_completos = []
    tiempo_inicio = time.time()
    
    for i, imagen_path in enumerate(imagenes_a_procesar):
        if mostrar_progreso:
            progreso = (i + 1) / len(imagenes_a_procesar) * 100
            print(f"\\n[{progreso:.1f}%] Procesando {imagen_path.name}...")
        
        # Ejecutar ambos pipelines
        resultado_rgb = ejecutar_pipeline_hibrido(imagen_path, usar_solo_rgb=True)
        resultado_hibrido = ejecutar_pipeline_hibrido(imagen_path, usar_solo_rgb=False)
        
        # Seleccionar el mejor resultado
        if resultado_rgb and resultado_hibrido:
            rgb_veg = np.sum(resultado_rgb['labels'] == 2) / resultado_rgb['labels'].size
            hyb_veg = np.sum(resultado_hibrido['labels'] == 2) / resultado_hibrido['labels'].size
            
            if hyb_veg > rgb_veg and hyb_veg > 0.05:
                mejor_resultado = resultado_hibrido
            else:
                mejor_resultado = resultado_rgb
            
            resultados_completos.append(mejor_resultado)
            print(f"  → Mejor método: {mejor_resultado['method']}")
        else:
            print(f"  ⚠️ Error al procesar {imagen_path.name}")

    tiempo_total = time.time() - tiempo_inicio
    print(f"\\n✅ Procesamiento completado en {tiempo_total:.1f} segundos.")
    
    return resultados_completos

# Procesar todas las imágenes
todos_los_resultados = procesar_y_guardar_resultados()

# Visualización en formato de línea de tiempo
if todos_los_resultados:
    # Ordenar resultados por año
    todos_los_resultados.sort(key=lambda x: x['year'])
    
    num_imagenes = len(todos_los_resultados)
    cols = 5  # Número de columnas en la visualización
    rows = (num_imagenes + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows), constrained_layout=True)
    axes = axes.flatten()

    for i, resultado in enumerate(todos_los_resultados):
        ax = axes[i]
        
        # Crear mapa de colores
        mapa_coloreado = crear_mapa_coloreado(resultado['labels'], class_colors)
        
        ax.imshow(mapa_coloreado)
        ax.set_title(f"Año: {resultado['year']}\\nMétodo: {resultado['method']}", fontsize=10)
        ax.axis('off')
        
        # Añadir estadísticas de distribución
        stats_text = []
        for j, name in enumerate(class_names, 1):
            percentage = 100.0 * np.sum(resultado['labels'] == j) / resultado['labels'].size
            stats_text.append(f"{name[0]}: {percentage:.1f}%")
        
        ax.text(0.5, -0.1, " | ".join(stats_text), 
                ha='center', va='center', transform=ax.transAxes, fontsize=9)

    # Ocultar ejes no utilizados
    for i in range(num_imagenes, len(axes)):
        axes[i].axis('off')

    fig.suptitle("Evolución de la Cobertura del Suelo a lo Largo del Tiempo", fontsize=24, fontweight='bold')
    
    # Crear leyenda global
    legend_elements = [Patch(facecolor=color, edgecolor='k', label=name) for name, color in zip(class_names, class_colors)]
    fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=14, bbox_to_anchor=(0.5, -0.02))
    
    plt.show()
    
    # Guardar la figura
    timeline_path = figures_path / "evolucion_temporal_segmentacion.png"
    fig.savefig(timeline_path, dpi=300, bbox_inches='tight')
    print(f"\\n🖼️  Visualización de la evolución guardada en: {timeline_path}")
else:
    print("No se generaron resultados para visualizar.")